### <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods, which will be used to get training and validation data loader.

You need to write a custom dataset class to load data.

**Note; There is   no separate validation data. , You will thus have to create your own validation set, by dividing the train data into train and validation data. Usually, we do 80:20 ratio for train and validation, respectively.**


For example:

```python
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    

```


```python
def get_data(args1, *args):
    ....
    ....
    return train_loader, test_loader
```

In [ ]:
#!/usr/bin/env python3
#global flag to indicate if the script is running in a local environment
g_local_run: bool = True

In [ ]:
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as F

import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics.classification import  MulticlassAccuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall
from torchmetrics import MeanMetric



import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd

from PIL import Image


In [ ]:
class KenyanFood13Dataset(Dataset):

    """Custom Dataset for Kenyan Food 13 Classification Task"""
    """ Accepts a CSV file with image ID and lable colums,
    Will split total images into train and validation sets with 80:20 ratio
    First 80% images will be used for training and remaining 20% for validation
    Args:
        annotations_file (string): Path to the csv file with annotations.
        img_dir (string): Directory with all the images.
        train (bool, optional): Indicates if the dataset is for training or validation.
            Default is True for training set, False for validation set.
        transform (callable, optional): Optional transform to be applied
            on a sample.
        target_transform (callable, optional): Optional transform to be applied
    """
    def __init__(self, annotations_file, img_dir, train=True, transform=None, target_transform=None):

        if g_local_run:
            print("Running in local mode - loading data from local paths")
            #add 'local' to annotatins_file name
            base, ext = os.path.splitext(annotations_file)
            annotations_file = f"{base}_local{ext}"

        # few error checks
        if not os.path.exists(annotations_file):
            raise FileNotFoundError(f"Annotations file not found: {annotations_file}")
        if not os.path.exists(img_dir):
            raise FileNotFoundError(f"Image directory not found: {img_dir}")

        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.local_run = g_local_run

        num_classes = len(self.img_labels['label'].unique())
        self.num_classes = num_classes
        print(f"Dataset initialized with {len(self.img_labels)} samples belonging to {num_classes} classes.")
        #split into train and validation sets
        split_index = int(0.8 * len(self.img_labels))
        if train:
            self.img_labels = self.img_labels.iloc[:split_index].reset_index(drop=True)
            print(f"Using {len(self.img_labels)} samples for training.")
        else:
            self.img_labels = self.img_labels.iloc[split_index:].reset_index(drop=True)
            print(f"Using {len(self.img_labels)} samples for validation.")

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")
        label = self.img_labels.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

    def __len__(self):
        return len(self.img_labels)


## <font style="color:green">2. Configuration [5 Points]</font>

**Define your configuration here.**

For example:


```python
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/opencv-pytorch-project-2-classification-round-3" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [ ]:
# configurations
from dataclasses import dataclass


@dataclass
class TrainingConfiguration:
    batch_size: int = 32
    learning_rate: float = 0.001
    num_epochs: int = 10
    momentum: float = 0.9
    log_interval: int = 10
    random_seed: int = 42

    model_name: str = "googlenet" # base model we will use for transfer learning and fine-tuning
    pretrained: bool = True # use pretrained weights for the base model
    precision: str = "float32" # precision for training: float32, float16, bfloat16
    fine_tune_start: int = 5 # layer from which to start fine-tuning (1 means all layers, higher means fewer layers)


@dataclass
class DataConfiguration:
    if g_local_run
        annotations_file: str = "../data/kenyan-food-13/train.csv"
        img_dir: str = "../data/kenyan-food-13/images/images"
    else:
        annotations_file: str = "/kaggle/input/kenyan-food-13/train.csv"
        img_dir: str = "/kaggle/input/kenyan-food-13/images/images"

    input_size: int = 224 # input image size for the model
    num_workers: int = 4 # number of workers for data loading

@dataclass
class systemConfiguration:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    if g_local_run:
        output_dir: str = "./output" # directory to save model checkpoints and logs
    else:
        output_dir: str = "/kaggle/working/output" # directory to save model checkpoints and logs





In [ ]:
data_config = DataConfiguration()
train_config = TrainingConfiguration()
system_config = systemConfiguration()

## <font style="color:green">3. Evaluation Metric [10 Points]</font>

**Define methods or classes that will be used in model evaluation. For example, accuracy, f1-score etc.**

In [ ]:
#we will have methods to calculate accuracy, f1-score, precision, recall.


## <font style="color:green">4. Train and Validation [5 Points]</font>


**Write the methods or classes to be used for training and validation.**

In [ ]:
def training_validation(training_config: TrainingConfiguration,
                        data_config: DataConfiguration,
                        system_config: systemConfiguration):

    #random seed for reproducibility
    pl.seed_everything(training_config.random_seed)

    model = KenyanFood13Classifier(training_config, data_config.num_classes)
    data_module = ... # define your data module here

    checkpoint_callback = ModelCheckpoint(
        dirpath=system_config.output_dir,
        filename="{epoch}-{val_loss:.2f}",
        save_top_k=3,
        monitor="valid/acc",
        mode="max",
        auto_insert_metric_name=False,
        save_weights_only=True)

    early_stopping_callback = EarlyStopping(
        monitor="valid/acc",
        patience=3,
        mode="max")

    # Map precision string to PyTorch Lightning expected value
    precision_map = {
        "float32": 32,
        "float16": 16,
        "bfloat16": "bf16"
    }
    trainer_precision = precision_map.get(training_config.precision, 32)

    trainer = L.Trainer(
        max_epochs=training_config.num_epochs,
        accelerator=system_config.device,
        devices="auto",
        precision=trainer_precision,
        callbacks=[checkpoint_callback, early_stopping_callback],
        default_root_dir=system_config.output_dir,
        log_every_n_steps=training_config.log_interval
    )

    trainer.fit(model, datamodule=data_module)
    trainer.validate(model, datamodule=data_module)

    return model, data_module, checkpoint_callback


## <font style="color:green">5. Model [5 Points]</font>

**Define your model in this section.**

**You are allowed to use any pre-trained model.**

In [ ]:
# LightningModule, we will use GoogleNet as base model for transfer learning and fine-tuning.
import torchvision


class KenyanFood13Classifier(L.LightningModule):
    def __init__(self, training_config: TrainingConfiguration, num_classes: int):
        super(KenyanFood13Classifier, self).__init__()
        self.save_hyperparameters()

        # Load base model
        if training_config.model_name == "googlenet":
            self.model = torchvision.models.googlenet(pretrained=training_config.pretrained)
            # Replace the final layer
            self.model.fc = torch.nn.Linear(self.model.fc.in_features, num_classes)
        else:
            raise ValueError(f"Model {training_config.model_name} not supported.")

        self.criterion = torch.nn.CrossEntropyLoss()
        self.train_mean_loss = MeanMetric()
        self.val_mean_loss = MeanMetric()


        self.train_accuracy = MulticlassAccuracy(num_classes=num_classes, average='macro')
        self.val_accuracy = MulticlassAccuracy(num_classes=num_classes, average='macro')
        self.train_f1 = MulticlassF1Score(num_classes=num_classes, average='macro')
        self.val_f1 = MulticlassF1Score(num_classes=num_classes, average='macro')
        self.train_precision = MulticlassPrecision(num_classes=num_classes, average='macro')
        self.val_precision = MulticlassPrecision(num_classes=num_classes, average='macro')
        self.train_recall = MulticlassRecall(num_classes=num_classes, average='macro')
        self.val_recall = MulticlassRecall(num_classes=num_classes, average='macro')

        self.learning_rate = training_config.learning_rate
        self.momentum = training_config.momentum



    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # get data from batch images, labels
        images, labels = batch
        # predictions
        outputs = self(images)
        # calculate loss, uses cross-entropy loss
        loss = self.criterion(outputs, labels)
        self.train_mean_loss.update(loss)

        preds = torch.argmax(outputs, dim=1)
        self.train_accuracy.update(preds, labels)
        self.train_mean_loss.update(loss)
        self.train_precision.update(preds, labels)
        self.train_recall.update(preds, labels)
        self.train_f1.update(preds, labels)
        self.log('train/loss', self.train_mean_loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train/acc', self.train_accuracy, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def on_train_epoch_end(self) -> None:
        #update  epoch level metrics and reset
        self.log('train/precision', self.train_precision.compute(), on_epoch=True, prog_bar=True)
        self.log('train/recall', self.train_recall.compute(), on_epoch=True, prog_bar=True)
        self.log('train/f1', self.train_f1.compute(), on_epoch=True, prog_bar=True)
        self.log('step', self.current_epoch, on_epoch=True, prog_bar=True)

        return super().on_train_epoch_end()

    def validation_step(self, batch, batch_idx):
        # get data from batch images, labels
        images, labels = batch
        # predictions
        outputs = self(images)
        # calculate loss, uses cross-entropy loss
        loss = self.criterion(outputs, labels)
        self.val_mean_loss.update(loss)

        preds = torch.argmax(outputs, dim=1)
        self.val_accuracy.update(preds, labels)
        self.log('valid/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('valid/acc', self.val_accuracy, on_step=False, on_epoch=True, prog_bar=True)

    def on_validation_epoch_end(self) -> None:
        #update  epoch level metrics and reset
        self.log('valid/precision', self.val_precision.compute(), on_epoch=True, prog_bar=True)
        self.log('valid/recall', self.val_recall.compute(), on_epoch=True, prog_bar=True)
        self.log('valid/f1', self.val_f1.compute(), on_epoch=True, prog_bar=True)
        self.log('step', self.current_epoch, on_epoch=True, prog_bar=True)

        return super().on_validation_epoch_end()


    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate, momentum=self.momentum)
        return optimizer

## <font style="color:green">6. Utils [5 Points]</font>

**Define those methods or classes, which have  not been covered in the above sections.**

## <font style="color:green">7. Experiment [5 Points]</font>

**Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.**

## <font style="color:green">8. TensorBoard Log Link [5 Points]</font>

**Share your TensorBoard scalars logs link here You can also share (not mandatory) your GitHub link, if you have pushed this project in GitHub.**


Note: In light of the recent shutdown of tensorboard.dev, we have updated the submission requirements for your project. Instead of sharing a tensorboard.dev link, you are now required to upload your generated TensorBoard event files directly onto the lab. As an alternative, you may also include a screenshot of your TensorBoard output within your Jupyter notebook. This adjustment ensures that your data visualization and model training efforts are thoroughly documented and accessible for evaluation.

You are also welcome (and encouraged) to utilize alternative logging services like wandB or comet. In such instances, you can easily make your project logs publicly accessible and share the link with others.

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

**Share your Kaggle profile link  with us here to score , points in  the competition.**

**For full points, you need a minimum accuracy of `75%` on the test data. If accuracy is less than `70%`, you gain  no points for this section.**


**Submit `submission.csv` (prediction for images in `test.csv`), in the `Submit Predictions` tab in Kaggle, to get evaluated for  this section.**